In [ ]:
to do 

find new datasets
decide on splitting ratio 20 60 20 




İMPLEMENTASYON

farklı embeddingler hangisinde en iyi sonucu alacağız, simiilarity olarak
classification için farklı şeyler düşünülebilir

farklı verisetleri




farklı similarity yöntemleri de kullanılabilir
	farklı yöntemler de dene
	cosine
	euclidean



40 40 20 yerine 
20 60 20

genel implementasyonu yap, sonuç al toy problemde
genel yöntemi çıkar - şema diyagram olarak
hangi aşamalar tamam
hangi aşamalarda çalışıyoruz
sorunlu olanları çıkar
research yapılması gerekenleri çıkar


deniz hanım

altun hanım


In [ ]:
implementation steps

1. reading data and preprocessing
